In [1]:
from solution_code import TwoBody, TwoBodyTMat, Cubherm, ThreeBody, OBEpot
import numpy as np
import matplotlib.pyplot as plt



para=[700.0, 0.020167185806378923]

pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])
kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0,j12max=1,l3max=0,j3min=1,
         j3max=1,parimin=1,parimax=1,tau3min=1,tau3max=1)
ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.05,e2=-0.06,elow=-0.02,tol=1e-8)
print("Ener:",ener*kernel.hbarc,eta)

nalphaNN 2
   alpha12    l12     s12      J12     t12
     0       0        0           0      1
     1       0        1           1      0
numabarNN 2
nalpha3N:  2
tau3 1
tau3 1
   alpha3N      l12       s12       J12          l3        I3        pari      t12     tau3
         0        0         0         0        0         1        1        1        1        1       1
         1        0         1         1        0         1        1        1        1        0       1
time:  38.085542200000006
pmat shape (2048, 2048)
E -0.05
maxpos, eigb 350 1.4183844112702273
E -0.06
maxpos, eigb 350 1.2928956292329634
E -0.08334038345722312
maxpos, eigb 350 1.0955406294444758
E -0.09320695484791053
maxpos, eigb 350 1.0353619921449
E -0.09900472022896614
maxpos, eigb 350 1.004204492523465
E -0.09978708920894304
maxpos, eigb 350 1.0002047010819746
E -0.09982712924079153
maxpos, eigb 350 1.0000012358222177
E -0.09982737243887367
maxpos, eigb 350 1.0000000003651146
E -0.099827372510746
maxpos, eigb 3

In [5]:
ekin[0]*kernel.hbarc
ener

-0.099827372510746

In [2]:
# fadout = kernel.faddeev(fadcomp, ener)
wf = kernel.applyperm(fadcomp)
ekin = kernel.calc_ekin(fadcomp, wf)
# epot = kernel.calc_vpot(fadout, wf)

print("Ener:",ener*kernel.hbarc,eta)
print(ekin)

Ener: -19.698635935427976 1.0000000000000004
(0.06544334715289304, 0.12547880284377794)


In [8]:
bj_list = [0,1]  # J=1/2, J=3/2
wf_list = []

pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])


for bj_idx, bj in enumerate(bj_list):
    print("Current bj: ", bj)
    # perform wavefunction evaluation for each angular momentum state
    kernel=ThreeBody(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=bj+1,j12max=bj,l3max=0,j3min=1,
         j3max=1,parimin=1,parimax=1,tau3min=1,tau3max=1)
    
    ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-3)
    # eigv, fadcomp = kernel.eigv_iter(E=-0.0047133078773, neigv=1)
    # eigv, fadcomp = kernel.eigv_iter(E=-0.099827372510746, neigv=1)
#     ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.05,e2=-0.06,elow=-0.001,tol=1e-5)
#     fadout = kernel.faddeev(fadcomp)
    wf = kernel.applyperm(fadcomp)
    
    # evaluate kinetic energy
    ekin = kernel.calc_ekin(fadcomp, wf)[0] * kernel.hbarc
    
    result_dict = {"wf":wf, "kernel":kernel, "ekin":ekin}
    wf_list.append(result_dict)

Current bj:  0
nalphaNN 2
   alpha12    l12     s12      J12     t12
     0       0        0           0      1
     1       1        1           0      1
numabarNN 2
nalpha3N:  1
tau3 1
   alpha3N      l12       s12       J12          l3        I3        pari      t12     tau3
         0        0         0         0        0         1        1        1        1        1       1
E -0.0046
maxpos, eigb 175 0.6740058961895672
E -0.0048
maxpos, eigb 175 0.6332836050535597
Current bj:  1
nalphaNN 6
   alpha12    l12     s12      J12     t12
     0       0        0           0      1
     1       1        1           0      1
     2       1        0           1      0
     3       1        1           1      1
     4       0        1           1      0
     5       2        1           1      0
numabarNN 8
nalpha3N:  3
tau3 1
tau3 1
tau3 1
   alpha3N      l12       s12       J12          l3        I3        pari      t12     tau3
         0        0         0         0        0         1   

In [9]:
for bj_idx, bj in enumerate(bj_list):
    # momentum grids are the same for each kernel, so can just use any of them
    k = wf_list[bj_idx]["kernel"]
    pgrid = k.pgrid
    qgrid = k.qgrid
    # alpha_idces = [0,1,10,-1]

    


    for alpha_dict in k.qnalpha:
        # quantum numbers associated to this particular alpha index
        # alpha_list = k.qnalpha[alpha_idx]
        # print(alpha_list)
        alpha_idx = alpha_dict["alpha"]
        qm_numbers = [alpha_dict["j"], alpha_dict["j3"], alpha_dict["t"]]

        # create combined grid
        P, Q = np.meshgrid(pgrid, qgrid)

        # plotting the results
        wf = np.abs(wf_list[bj_idx]["wf"][alpha_idx,:,:])  # take abs value
        
        fig, ax = plt.subplots(figsize=(8,8))
        im = ax.contourf(P, Q, wf, vmin=np.min(wf), vmax=np.max(wf), cmap=plt.cm.YlGnBu_r)
        ax.set_xlabel("p / fm$^{{-1}}$", fontsize=12)
        ax.set_ylabel("q / fm$^{{-1}}$", fontsize=12)
        if bj_idx == 0:
            ax.set_xlim([0,1.2])
            ax.set_ylim([0,1.2])
        ax.set_title("Wavefunction with $j_{{12}}={0}, j_3={1}, t_{{12}}={2}$ for $J={3}$".format(*qm_numbers, bj))
        
        cbar = fig.colorbar(im)
        cbar.ax.set_ylabel("$|\Psi|$ / fm$^3$", fontsize=14)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices